<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2E_Bloque_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
================================================================================
FASE 2E - BLOQUE 1: VISUALIZACIONES ESTADISTICAS
================================================================================

Trabajo Fin de Master - Evaluacion Comparativa de Tecnicas de Segmentacion
                        en Fotografia de Retrato

Autor: Jesus L.
Universidad: Universitat Oberta de Catalunya (UOC)
Master: Data Science
Fecha: Diciembre 2025

Descripcion:
    Generacion de 13 visualizaciones estadisticas para el Capitulo 4 del TFM:

    1. Box plot IoU por modelo
    2. Violin plot IoU por modelo
    3. Heatmap IoU modelo x fotografia
    4. Scatter apertura vs IoU
    5. Scatter complejidad fondo vs IoU
    6. Scatter contraste vs IoU
    7. Radar chart perfil por modelo
    8. Bar horizontal ranking TOP-20
    9. Line plot IoU por fotografia
    10. Heatmap matriz de correlaciones
    11. PCA 3D scatter
    12. Box plot por arquitectura
    13. Sensibilidad a umbrales

Uso:
    from analisis_fase_2e_bloque1 import ejecutar_bloque1
    resultados = ejecutar_bloque1()

================================================================================
"""

'\n================================================================================\nFASE 2E - BLOQUE 1: VISUALIZACIONES ESTADISTICAS\n================================================================================\n\nTrabajo Fin de Master - Evaluacion Comparativa de Tecnicas de Segmentacion\n                        en Fotografia de Retrato\n\nAutor: Jesus L.\nUniversidad: Universitat Oberta de Catalunya (UOC)\nMaster: Data Science\nFecha: Diciembre 2025\n\nDescripcion:\n    Generacion de 13 visualizaciones estadisticas para el Capitulo 4 del TFM:\n    \n    1. Box plot IoU por modelo\n    2. Violin plot IoU por modelo\n    3. Heatmap IoU modelo x fotografia\n    4. Scatter apertura vs IoU\n    5. Scatter complejidad fondo vs IoU\n    6. Scatter contraste vs IoU\n    7. Radar chart perfil por modelo\n    8. Bar horizontal ranking TOP-20\n    9. Line plot IoU por fotografia\n    10. Heatmap matriz de correlaciones\n    11. PCA 3D scatter\n    12. Box plot por arquitectura\n    13. Sens

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
!ls -la /content/drive/MyDrive/TFM/3_Analisis/fase2e_visualizaciones/

total 40
-rw------- 1 root root 19727 Dec 12 23:43 analisis_fase_2e_config.py
drwx------ 2 root root  4096 Dec 12 23:47 bloque1_estadisticas
drwx------ 2 root root  4096 Dec 12 23:43 bloque2_dashboards
drwx------ 2 root root  4096 Dec 12 23:43 bloque3_mascaras
drwx------ 2 root root  4096 Dec 12 23:43 bloque4_catalogo
drwx------ 2 root root  4096 Dec 12 23:43 __pycache__


In [27]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/TFM/3_Analisis/fase2e_visualizaciones')

In [28]:
from analisis_fase_2e_config import inicializar_fase2e
estado = inicializar_fase2e()

[23:56:16] INFO - ============================================================


INFO:Fase2E_Init:============================================================


[23:56:16] INFO - INICIALIZANDO FASE 2E - VISUALIZACIONES


INFO:Fase2E_Init:INICIALIZANDO FASE 2E - VISUALIZACIONES


[23:56:16] INFO - ============================================================


...

INFO:Fase2E_Init:  anova_por_modelo: OK


[23:56:16] INFO -   indice_maestro: OK


INFO:Fase2E_Init:  indice_maestro: OK


[23:56:16] INFO - Archivos disponibles: 10/10


INFO:Fase2E_Init:Archivos disponibles: 10/10


In [29]:
# =============================================================================
# IMPORTACIONES
# =============================================================================

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import seaborn as sns
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from mpl_toolkits.mplot3d import Axes3D

# Importar configuracion compartida
try:
    from analisis_fase_2e_config import (
        RUTAS, ARCHIVOS, FIGSIZE, EXPORT_CONFIG,
        PALETTE_MODELS, ORDEN_MODELOS, NOMBRES_MODELOS,
        PALETTE_ARQUITECTURAS, PALETTE_CLUSTERS,
        configurar_logging, aplicar_estilo, guardar_figura,
        cargar_datos_metricas, cargar_correlaciones, cargar_clusters, cargar_pca,
        agregar_columna_modelo, obtener_color_modelo, obtener_nombre_modelo
    )
except ImportError:
    # Para ejecucion directa en Colab
    exec(open('analisis_fase_2e_config.py').read())

In [30]:
# =============================================================================
# VISUALIZACION 1: BOX PLOT IoU POR MODELO
# =============================================================================

def viz_01_boxplot_iou_modelos(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera box plot de distribucion de IoU por modelo.

    Muestra la distribucion, mediana, cuartiles y outliers
    para cada modelo de segmentacion.

    Args:
        df: DataFrame con metricas (columnas: modelo_norm, iou)
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Box plot IoU por modelo")

    fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])

    # Preparar datos
    df_plot = df[df['modelo_norm'].isin(ORDEN_MODELOS)].copy()

    # Crear box plot
    colores = [PALETTE_MODELS.get(m, '#7F8C8D') for m in ORDEN_MODELOS]

    box = ax.boxplot(
        [df_plot[df_plot['modelo_norm'] == m]['iou'].dropna() for m in ORDEN_MODELOS],
        labels=[NOMBRES_MODELOS.get(m, m) for m in ORDEN_MODELOS],
        patch_artist=True,
        medianprops={'color': 'black', 'linewidth': 1.5},
        flierprops={'marker': 'o', 'markersize': 4, 'alpha': 0.5},
        whiskerprops={'linewidth': 1.2},
        capprops={'linewidth': 1.2}
    )

    # Colorear cajas
    for patch, color in zip(box['boxes'], colores):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)

    # Configuracion
    ax.set_ylabel('IoU (Intersection over Union)', fontsize=11)
    ax.set_xlabel('Modelo de Segmentacion', fontsize=11)
    ax.set_title('Distribucion de IoU por Modelo de Segmentacion', fontsize=13, fontweight='bold')
    ax.set_ylim(0, 1.05)
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)

    # Rotar etiquetas si es necesario
    plt.xticks(rotation=15, ha='right')

    # Agregar linea de referencia
    ax.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5, label='IoU = 0.5')

    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_01_boxplot_iou_modelos', output_dir, logger=logger)

    return fig

In [31]:
# =============================================================================
# VISUALIZACION 2: VIOLIN PLOT IoU POR MODELO
# =============================================================================

def viz_02_violin_iou_modelos(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera violin plot de distribucion de IoU por modelo.

    Muestra la densidad de la distribucion ademas de los cuartiles.

    Args:
        df: DataFrame con metricas
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Violin plot IoU por modelo")

    fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])

    # Preparar datos
    df_plot = df[df['modelo_norm'].isin(ORDEN_MODELOS)].copy()
    df_plot['modelo_label'] = df_plot['modelo_norm'].map(NOMBRES_MODELOS)

    # Ordenar categorias
    orden_labels = [NOMBRES_MODELOS[m] for m in ORDEN_MODELOS if m in df_plot['modelo_norm'].unique()]

    # Crear violin plot
    palette = {NOMBRES_MODELOS[m]: PALETTE_MODELS[m] for m in ORDEN_MODELOS}

    sns.violinplot(
        data=df_plot,
        x='modelo_label',
        y='iou',
        order=orden_labels,
        palette=palette,
        inner='quartile',
        ax=ax,
        alpha=0.8
    )

    # Configuracion
    ax.set_ylabel('IoU (Intersection over Union)', fontsize=11)
    ax.set_xlabel('Modelo de Segmentacion', fontsize=11)
    ax.set_title('Densidad de Distribucion de IoU por Modelo', fontsize=13, fontweight='bold')
    ax.set_ylim(0, 1.05)

    plt.xticks(rotation=15, ha='right')
    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_02_violin_iou_modelos', output_dir, logger=logger)

    return fig

In [32]:
# =============================================================================
# VISUALIZACION 3: HEATMAP IoU MODELO x FOTOGRAFIA
# =============================================================================

def viz_03_heatmap_iou_modelo_foto(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera heatmap de IoU por modelo y fotografia.

    Permite identificar patrones de rendimiento y fotografias problematicas.

    Args:
        df: DataFrame con metricas
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Heatmap IoU modelo x fotografia")

    # Detectar nombre de columna de foto
    col_foto = 'codigo_foto' if 'codigo_foto' in df.columns else 'foto_id'

    # Pivotar datos: filas=fotos, columnas=modelos
    # Primero agregar por modelo (mejor config por modelo/foto)
    df_agg = df.groupby([col_foto, 'modelo_norm'])['iou'].max().reset_index()

    df_pivot = df_agg.pivot(
        index=col_foto,
        columns='modelo_norm',
        values='iou'
    )

    # Ordenar columnas segun ORDEN_MODELOS
    cols_ordenadas = [m for m in ORDEN_MODELOS if m in df_pivot.columns]
    df_pivot = df_pivot[cols_ordenadas]

    # Renombrar columnas
    df_pivot.columns = [NOMBRES_MODELOS.get(c, c) for c in df_pivot.columns]

    # Ordenar filas por IoU promedio
    df_pivot['mean'] = df_pivot.mean(axis=1)
    df_pivot = df_pivot.sort_values('mean', ascending=False)
    df_pivot = df_pivot.drop('mean', axis=1)

    # Crear figura
    fig, ax = plt.subplots(figsize=FIGSIZE['grande'])

    # Heatmap
    sns.heatmap(
        df_pivot,
        annot=True,
        fmt='.2f',
        cmap='RdYlGn',
        vmin=0,
        vmax=1,
        center=0.5,
        ax=ax,
        cbar_kws={'label': 'IoU', 'shrink': 0.8},
        annot_kws={'size': 8}
    )

    # Configuracion
    ax.set_xlabel('Modelo de Segmentacion', fontsize=11)
    ax.set_ylabel('Fotografia', fontsize=11)
    ax.set_title('Rendimiento IoU por Modelo y Fotografia\n(Mejor configuracion por modelo)',
                 fontsize=13, fontweight='bold')

    plt.xticks(rotation=30, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_03_heatmap_iou_modelo_foto', output_dir, logger=logger)

    return fig

In [33]:
# =============================================================================
# VISUALIZACION 4: SCATTER APERTURA vs IoU
# =============================================================================

def viz_04_scatter_apertura_iou(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera scatter plot de apertura (f-number) vs IoU.

    Evalua la hipotesis de que mayor bokeh (menor f-number) mejora segmentacion.

    Args:
        df: DataFrame con metricas y caracteristicas fotograficas
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Scatter apertura vs IoU")

    fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])

    # Verificar columna de apertura
    col_apertura = None
    for col in ['exif_apertura', 'apertura_fnumber', 'fnumber', 'apertura', 'exif_apertura_fnumber']:
        if col in df.columns:
            col_apertura = col
            break

    if col_apertura is None:
        if logger:
            logger.warning("No se encuentra columna de apertura")
        ax.text(0.5, 0.5, 'Datos de apertura no disponibles',
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        guardar_figura(fig, 'viz_04_scatter_apertura_iou', output_dir, logger=logger)
        return fig

    # Filtrar datos validos
    df_plot = df[[col_apertura, 'iou', 'modelo_norm']].dropna()

    # Scatter por modelo
    for modelo in ORDEN_MODELOS:
        df_m = df_plot[df_plot['modelo_norm'] == modelo]
        if len(df_m) > 0:
            ax.scatter(
                df_m[col_apertura],
                df_m['iou'],
                c=PALETTE_MODELS.get(modelo, '#7F8C8D'),
                label=NOMBRES_MODELOS.get(modelo, modelo),
                alpha=0.6,
                s=50,
                edgecolor='white',
                linewidth=0.5
            )

    # Linea de tendencia global
    if len(df_plot) > 10:
        z = np.polyfit(df_plot[col_apertura], df_plot['iou'], 1)
        p = np.poly1d(z)
        x_line = np.linspace(df_plot[col_apertura].min(), df_plot[col_apertura].max(), 100)
        ax.plot(x_line, p(x_line), 'k--', alpha=0.5, linewidth=2, label='Tendencia')

        # Calcular correlacion
        corr = df_plot[col_apertura].corr(df_plot['iou'])
        ax.annotate(f'r = {corr:.3f}', xy=(0.95, 0.05), xycoords='axes fraction',
                   ha='right', fontsize=10, style='italic')

    # Configuracion
    ax.set_xlabel('Apertura (f-number)', fontsize=11)
    ax.set_ylabel('IoU', fontsize=11)
    ax.set_title('Relacion entre Apertura y Rendimiento de Segmentacion\n(Hipotesis: Bokeh mejora segmentacion)',
                 fontsize=13, fontweight='bold')
    ax.legend(loc='upper left', fontsize=8, ncol=2)
    ax.set_ylim(0, 1.05)

    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_04_scatter_apertura_iou', output_dir, logger=logger)

    return fig

In [34]:
# =============================================================================
# VISUALIZACION 5: SCATTER COMPLEJIDAD FONDO vs IoU
# =============================================================================

def viz_05_scatter_complejidad_iou(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera scatter plot de complejidad del fondo vs IoU.

    Evalua la hipotesis de que fondos complejos dificultan segmentacion.

    Args:
        df: DataFrame con metricas y caracteristicas fotograficas
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Scatter complejidad fondo vs IoU")

    fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])

    # Buscar columna de complejidad
    col_complejidad = None
    for col in ['tex_haralick_entropy', 'color_entropia', 'haralick_entropy', 'entropia', 'textura_entropia', 'glcm_entropy']:
        if col in df.columns:
            col_complejidad = col
            break

    if col_complejidad is None:
        if logger:
            logger.warning("No se encuentra columna de complejidad")
        ax.text(0.5, 0.5, 'Datos de complejidad no disponibles',
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        guardar_figura(fig, 'viz_05_scatter_complejidad_iou', output_dir, logger=logger)
        return fig

    # Filtrar datos validos
    df_plot = df[[col_complejidad, 'iou', 'modelo_norm']].dropna()

    # Scatter por modelo
    for modelo in ORDEN_MODELOS:
        df_m = df_plot[df_plot['modelo_norm'] == modelo]
        if len(df_m) > 0:
            ax.scatter(
                df_m[col_complejidad],
                df_m['iou'],
                c=PALETTE_MODELS.get(modelo, '#7F8C8D'),
                label=NOMBRES_MODELOS.get(modelo, modelo),
                alpha=0.6,
                s=50,
                edgecolor='white',
                linewidth=0.5
            )

    # Linea de tendencia
    if len(df_plot) > 10:
        z = np.polyfit(df_plot[col_complejidad], df_plot['iou'], 1)
        p = np.poly1d(z)
        x_line = np.linspace(df_plot[col_complejidad].min(), df_plot[col_complejidad].max(), 100)
        ax.plot(x_line, p(x_line), 'k--', alpha=0.5, linewidth=2, label='Tendencia')

        corr = df_plot[col_complejidad].corr(df_plot['iou'])
        ax.annotate(f'r = {corr:.3f}', xy=(0.95, 0.05), xycoords='axes fraction',
                   ha='right', fontsize=10, style='italic')

    # Configuracion
    ax.set_xlabel('Entropia Textural (Complejidad del Fondo)', fontsize=11)
    ax.set_ylabel('IoU', fontsize=11)
    ax.set_title('Relacion entre Complejidad del Fondo y Rendimiento\n(Hipotesis: Fondos complejos dificultan segmentacion)',
                 fontsize=13, fontweight='bold')
    ax.legend(loc='upper right', fontsize=8, ncol=2)
    ax.set_ylim(0, 1.05)

    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_05_scatter_complejidad_iou', output_dir, logger=logger)

    return fig

In [35]:
# =============================================================================
# VISUALIZACION 6: SCATTER CONTRASTE vs IoU
# =============================================================================

def viz_06_scatter_contraste_iou(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera scatter plot de contraste vs IoU.

    Evalua la hipotesis de que mayor contraste mejora segmentacion.

    Args:
        df: DataFrame con metricas y caracteristicas fotograficas
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Scatter contraste vs IoU")

    fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])

    # Buscar columna de contraste
    col_contraste = None
    for col in ['contraste_rms', 'contraste', 'rms_contraste', 'calidad_contraste_rms']:
        if col in df.columns:
            col_contraste = col
            break

    if col_contraste is None:
        if logger:
            logger.warning("No se encuentra columna de contraste")
        ax.text(0.5, 0.5, 'Datos de contraste no disponibles',
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        guardar_figura(fig, 'viz_06_scatter_contraste_iou', output_dir, logger=logger)
        return fig

    # Filtrar datos validos
    df_plot = df[[col_contraste, 'iou', 'modelo_norm']].dropna()

    # Scatter por modelo
    for modelo in ORDEN_MODELOS:
        df_m = df_plot[df_plot['modelo_norm'] == modelo]
        if len(df_m) > 0:
            ax.scatter(
                df_m[col_contraste],
                df_m['iou'],
                c=PALETTE_MODELS.get(modelo, '#7F8C8D'),
                label=NOMBRES_MODELOS.get(modelo, modelo),
                alpha=0.6,
                s=50,
                edgecolor='white',
                linewidth=0.5
            )

    # Linea de tendencia
    if len(df_plot) > 10:
        z = np.polyfit(df_plot[col_contraste], df_plot['iou'], 1)
        p = np.poly1d(z)
        x_line = np.linspace(df_plot[col_contraste].min(), df_plot[col_contraste].max(), 100)
        ax.plot(x_line, p(x_line), 'k--', alpha=0.5, linewidth=2, label='Tendencia')

        corr = df_plot[col_contraste].corr(df_plot['iou'])
        ax.annotate(f'r = {corr:.3f}', xy=(0.95, 0.05), xycoords='axes fraction',
                   ha='right', fontsize=10, style='italic')

    # Configuracion
    ax.set_xlabel('Contraste RMS', fontsize=11)
    ax.set_ylabel('IoU', fontsize=11)
    ax.set_title('Relacion entre Contraste y Rendimiento de Segmentacion\n(Hipotesis: Mayor contraste mejora segmentacion)',
                 fontsize=13, fontweight='bold')
    ax.legend(loc='best', fontsize=8, ncol=2)
    ax.set_ylim(0, 1.05)

    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_06_scatter_contraste_iou', output_dir, logger=logger)

    return fig

In [36]:
# =============================================================================
# VISUALIZACION 7: RADAR CHART PERFIL POR MODELO
# =============================================================================

def viz_07_radar_modelos(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera radar chart comparativo de metricas por modelo.

    Muestra perfil multidimensional de cada modelo.

    Args:
        df: DataFrame con metricas
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Radar chart perfil por modelo")

    # Metricas para el radar
    metricas_radar = ['iou', 'dice', 'precision', 'recall']
    metricas_disponibles = [m for m in metricas_radar if m in df.columns]

    if len(metricas_disponibles) < 3:
        if logger:
            logger.warning("Insuficientes metricas para radar chart")
        fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])
        ax.text(0.5, 0.5, 'Metricas insuficientes para radar chart',
                ha='center', va='center', transform=ax.transAxes)
        guardar_figura(fig, 'viz_07_radar_modelos', output_dir, logger=logger)
        return fig

    # Calcular promedios por modelo
    df_stats = df.groupby('modelo_norm')[metricas_disponibles].mean()

    # Filtrar modelos disponibles
    modelos_disponibles = [m for m in ORDEN_MODELOS if m in df_stats.index]
    df_stats = df_stats.loc[modelos_disponibles]

    # Configuracion del radar
    num_vars = len(metricas_disponibles)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]  # Cerrar el poligono

    # Crear figura
    fig, ax = plt.subplots(figsize=FIGSIZE['cuadrada'], subplot_kw=dict(polar=True))

    # Dibujar cada modelo
    for modelo in modelos_disponibles:
        valores = df_stats.loc[modelo, metricas_disponibles].values.tolist()
        valores += valores[:1]  # Cerrar

        color = PALETTE_MODELS.get(modelo, '#7F8C8D')
        ax.plot(angles, valores, 'o-', linewidth=2, color=color,
                label=NOMBRES_MODELOS.get(modelo, modelo))
        ax.fill(angles, valores, alpha=0.15, color=color)

    # Configuracion
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels([m.upper() for m in metricas_disponibles], fontsize=10)
    ax.set_ylim(0, 1)
    ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=8)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), fontsize=9)

    plt.title('Perfil de Metricas por Modelo de Segmentacion',
              fontsize=13, fontweight='bold', y=1.08)

    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_07_radar_modelos', output_dir, logger=logger)

    return fig

In [37]:
# =============================================================================
# VISUALIZACION 8: BAR HORIZONTAL RANKING TOP-20
# =============================================================================

def viz_08_ranking_top20(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera bar chart horizontal del ranking TOP-20 configuraciones.

    Args:
        df: DataFrame con metricas
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Ranking TOP-20 configuraciones")

    # Calcular IoU promedio por configuracion
    df_ranking = df.groupby('config_codigo').agg({
        'iou': 'mean',
        'modelo_norm': 'first'
    }).reset_index()

    df_ranking = df_ranking.sort_values('iou', ascending=False).head(20)

    # Crear figura
    fig, ax = plt.subplots(figsize=FIGSIZE['grande'])

    # Colores por modelo
    colores = [PALETTE_MODELS.get(m, '#7F8C8D') for m in df_ranking['modelo_norm']]

    # Bar chart horizontal
    y_pos = np.arange(len(df_ranking))
    bars = ax.barh(y_pos, df_ranking['iou'], color=colores, edgecolor='white', linewidth=0.5)

    # Etiquetas
    ax.set_yticks(y_pos)
    ax.set_yticklabels(df_ranking['config_codigo'], fontsize=8)
    ax.invert_yaxis()  # Mayor arriba

    # Valores en las barras
    for i, (bar, val) in enumerate(zip(bars, df_ranking['iou'])):
        ax.text(val + 0.01, bar.get_y() + bar.get_height()/2,
                f'{val:.4f}', va='center', fontsize=8)

    # Leyenda de modelos
    handles = [Patch(facecolor=PALETTE_MODELS[m], label=NOMBRES_MODELOS[m])
               for m in ORDEN_MODELOS if m in df_ranking['modelo_norm'].values]
    ax.legend(handles=handles, loc='lower right', fontsize=8)

    # Configuracion
    ax.set_xlabel('IoU Promedio', fontsize=11)
    ax.set_ylabel('Configuracion', fontsize=11)
    ax.set_title('Ranking TOP-20 Configuraciones por IoU', fontsize=13, fontweight='bold')
    ax.set_xlim(0, 1.1)
    ax.xaxis.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_08_ranking_top20', output_dir, logger=logger)

    return fig

In [38]:
# =============================================================================
# VISUALIZACION 9: LINE PLOT IoU POR FOTOGRAFIA
# =============================================================================

def viz_09_lineas_iou_fotos(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera line plot de IoU por fotografia para todos los modelos.

    Permite ver variabilidad entre fotografias.

    Args:
        df: DataFrame con metricas
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Line plot IoU por fotografia")

    # Detectar nombre de columna de foto
    col_foto = 'codigo_foto' if 'codigo_foto' in df.columns else 'foto_id'

    # Agregar: mejor IoU por modelo y foto
    df_agg = df.groupby([col_foto, 'modelo_norm'])['iou'].max().reset_index()

    # Pivotar
    df_pivot = df_agg.pivot(index=col_foto, columns='modelo_norm', values='iou')

    # Ordenar por IoU promedio
    df_pivot['mean'] = df_pivot.mean(axis=1)
    df_pivot = df_pivot.sort_values('mean', ascending=False)
    df_pivot = df_pivot.drop('mean', axis=1)

    # Crear figura
    fig, ax = plt.subplots(figsize=FIGSIZE['extra_grande'])

    # Plotear cada modelo
    x = np.arange(len(df_pivot))

    for modelo in ORDEN_MODELOS:
        if modelo in df_pivot.columns:
            ax.plot(x, df_pivot[modelo], 'o-',
                   color=PALETTE_MODELS.get(modelo, '#7F8C8D'),
                   label=NOMBRES_MODELOS.get(modelo, modelo),
                   linewidth=1.5, markersize=5, alpha=0.8)

    # Configuracion
    ax.set_xticks(x)
    ax.set_xticklabels(df_pivot.index, rotation=45, ha='right', fontsize=8)
    ax.set_xlabel('Fotografia', fontsize=11)
    ax.set_ylabel('IoU', fontsize=11)
    ax.set_title('Rendimiento IoU por Fotografia y Modelo\n(Mejor configuracion por modelo)',
                 fontsize=13, fontweight='bold')
    ax.legend(loc='lower left', fontsize=9, ncol=2)
    ax.set_ylim(0, 1.05)
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_09_lineas_iou_fotos', output_dir, logger=logger)

    return fig

In [39]:
# =============================================================================
# VISUALIZACION 10: HEATMAP CORRELACIONES
# =============================================================================

def viz_10_heatmap_correlaciones(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera heatmap de matriz de correlaciones entre metricas.

    Args:
        df: DataFrame con metricas
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Heatmap matriz de correlaciones")

    # Seleccionar columnas numericas relevantes
    metricas = ['iou', 'dice', 'precision', 'recall', 'boundary_iou',
                'area', 'perimeter', 'compactness', 'convexity', 'solidity']

    cols_disponibles = [c for c in metricas if c in df.columns]

    if len(cols_disponibles) < 3:
        if logger:
            logger.warning("Insuficientes columnas para correlaciones")
        fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])
        ax.text(0.5, 0.5, 'Datos insuficientes para correlaciones',
                ha='center', va='center', transform=ax.transAxes)
        guardar_figura(fig, 'viz_10_heatmap_correlaciones', output_dir, logger=logger)
        return fig

    # Calcular correlaciones
    df_corr = df[cols_disponibles].corr()

    # Crear figura
    fig, ax = plt.subplots(figsize=FIGSIZE['grande'])

    # Heatmap
    mask = np.triu(np.ones_like(df_corr, dtype=bool), k=1)

    sns.heatmap(
        df_corr,
        mask=mask,
        annot=True,
        fmt='.2f',
        cmap='RdBu_r',
        vmin=-1,
        vmax=1,
        center=0,
        ax=ax,
        square=True,
        cbar_kws={'label': 'Correlacion', 'shrink': 0.8},
        annot_kws={'size': 9}
    )

    # Configuracion
    ax.set_title('Matriz de Correlaciones entre Metricas de Segmentacion',
                 fontsize=13, fontweight='bold')

    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_10_heatmap_correlaciones', output_dir, logger=logger)

    return fig

In [40]:
# =============================================================================
# VISUALIZACION 11: PCA 3D SCATTER
# =============================================================================

def viz_11_pca_3d_scatter(
    df: pd.DataFrame,
    df_pca: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera scatter 3D de componentes PCA.

    Args:
        df: DataFrame con metricas
        df_pca: DataFrame con componentes PCA
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: PCA 3D scatter")

    if df_pca is None or len(df_pca) == 0:
        if logger:
            logger.warning("Datos PCA no disponibles")
        fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])
        ax.text(0.5, 0.5, 'Datos PCA no disponibles',
                ha='center', va='center', transform=ax.transAxes)
        guardar_figura(fig, 'viz_11_pca_3d_scatter', output_dir, logger=logger)
        return fig

    # Verificar columnas PCA
    pca_cols = ['PC1', 'PC2', 'PC3']
    if not all(c in df_pca.columns for c in pca_cols):
        # Buscar alternativas
        pca_cols = [c for c in df_pca.columns if c.startswith('PC') or c.startswith('pca')][:3]

    if len(pca_cols) < 3:
        if logger:
            logger.warning("Insuficientes componentes PCA")
        fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])
        ax.text(0.5, 0.5, 'Insuficientes componentes PCA',
                ha='center', va='center', transform=ax.transAxes)
        guardar_figura(fig, 'viz_11_pca_3d_scatter', output_dir, logger=logger)
        return fig

    # Crear figura 3D
    fig = plt.figure(figsize=FIGSIZE['grande'])
    ax = fig.add_subplot(111, projection='3d')

    # Agregar columna de modelo si no existe
    if 'modelo_norm' not in df_pca.columns:
        if 'config_codigo' in df_pca.columns:
            df_pca = agregar_columna_modelo(df_pca.copy())

    # Scatter por modelo
    for modelo in ORDEN_MODELOS:
        if 'modelo_norm' in df_pca.columns:
            mask = df_pca['modelo_norm'] == modelo
            df_m = df_pca[mask]
        else:
            df_m = df_pca

        if len(df_m) > 0:
            ax.scatter(
                df_m[pca_cols[0]],
                df_m[pca_cols[1]],
                df_m[pca_cols[2]],
                c=PALETTE_MODELS.get(modelo, '#7F8C8D'),
                label=NOMBRES_MODELOS.get(modelo, modelo),
                alpha=0.7,
                s=50,
                edgecolor='white',
                linewidth=0.3
            )

    # Configuracion
    ax.set_xlabel(pca_cols[0], fontsize=10)
    ax.set_ylabel(pca_cols[1], fontsize=10)
    ax.set_zlabel(pca_cols[2], fontsize=10)
    ax.legend(loc='upper left', fontsize=8)

    plt.title('Proyeccion PCA de Configuraciones de Segmentacion',
              fontsize=13, fontweight='bold')

    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_11_pca_3d_scatter', output_dir, logger=logger)

    return fig

In [41]:
# =============================================================================
# VISUALIZACION 12: BOX PLOT POR ARQUITECTURA
# =============================================================================

def viz_12_boxplot_arquitecturas(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera box plot de IoU agrupado por tipo de arquitectura.

    Compara CNN vs Transformer vs Foundation vs Web.

    Args:
        df: DataFrame con metricas
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Box plot por arquitectura")

    # Mapeo de modelo a arquitectura
    arquitectura_map = {
        'yolov8': 'CNN',
        'oneformer': 'Transformer',
        'mask2former': 'Transformer',
        'sam2_prompts': 'Foundation',
        'sam2_auto': 'Foundation',
        'bodypix': 'Web',
    }

    df_plot = df.copy()
    df_plot['arquitectura'] = df_plot['modelo_norm'].map(arquitectura_map)
    df_plot = df_plot.dropna(subset=['arquitectura'])

    # Orden de arquitecturas
    orden_arq = ['CNN', 'Transformer', 'Foundation', 'Web']
    orden_arq = [a for a in orden_arq if a in df_plot['arquitectura'].unique()]

    # Crear figura
    fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])

    # Box plot
    colores = [PALETTE_ARQUITECTURAS.get(a, '#7F8C8D') for a in orden_arq]

    box = ax.boxplot(
        [df_plot[df_plot['arquitectura'] == a]['iou'].dropna() for a in orden_arq],
        labels=orden_arq,
        patch_artist=True,
        medianprops={'color': 'black', 'linewidth': 1.5},
        flierprops={'marker': 'o', 'markersize': 4, 'alpha': 0.5}
    )

    for patch, color in zip(box['boxes'], colores):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)

    # Configuracion
    ax.set_ylabel('IoU', fontsize=11)
    ax.set_xlabel('Tipo de Arquitectura', fontsize=11)
    ax.set_title('Rendimiento IoU por Paradigma Arquitectonico',
                 fontsize=13, fontweight='bold')
    ax.set_ylim(0, 1.05)
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)

    # Agregar conteo
    for i, arq in enumerate(orden_arq):
        n = len(df_plot[df_plot['arquitectura'] == arq])
        ax.annotate(f'n={n}', xy=(i+1, 0.02), ha='center', fontsize=9, style='italic')

    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_12_boxplot_arquitecturas', output_dir, logger=logger)

    return fig

In [42]:
# =============================================================================
# VISUALIZACION 13: SENSIBILIDAD A UMBRALES
# =============================================================================

def viz_13_sensibilidad_umbrales(
    df: pd.DataFrame,
    output_dir: Path,
    logger=None
) -> plt.Figure:
    """
    Genera visualizacion de sensibilidad a umbrales de confianza.

    Args:
        df: DataFrame con metricas
        output_dir: Directorio de salida
        logger: Logger para mensajes

    Returns:
        Figura matplotlib
    """
    if logger:
        logger.info("Generando: Sensibilidad a umbrales")

    # Intentar extraer umbral del config_codigo
    def extraer_umbral(config):
        """Extrae umbral numerico del codigo de configuracion."""
        config = str(config).lower()
        # Buscar patrones como 'th_0.5', 'umbral_0.3', etc.
        import re
        match = re.search(r'(?:th|umbral|conf)[_]?(\d+\.?\d*)', config)
        if match:
            return float(match.group(1))
        # Buscar sensibilidad
        if 'high' in config or 'alta' in config:
            return 0.7
        elif 'low' in config or 'baja' in config:
            return 0.3
        elif 'medium' in config or 'media' in config or 'balanced' in config:
            return 0.5
        return None

    df_plot = df.copy()
    df_plot['umbral'] = df_plot['config_codigo'].apply(extraer_umbral)
    df_plot = df_plot.dropna(subset=['umbral'])

    if len(df_plot) < 10:
        if logger:
            logger.warning("Insuficientes datos con umbral identificable")
        fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])
        ax.text(0.5, 0.5, 'Insuficientes datos de umbrales identificables',
                ha='center', va='center', transform=ax.transAxes)
        guardar_figura(fig, 'viz_13_sensibilidad_umbrales', output_dir, logger=logger)
        return fig

    # Crear figura
    fig, ax = plt.subplots(figsize=FIGSIZE['mediana'])

    # Agregar por umbral y modelo
    df_agg = df_plot.groupby(['umbral', 'modelo_norm'])['iou'].mean().reset_index()

    # Plotear cada modelo
    for modelo in ORDEN_MODELOS:
        df_m = df_agg[df_agg['modelo_norm'] == modelo].sort_values('umbral')
        if len(df_m) > 1:
            ax.plot(
                df_m['umbral'],
                df_m['iou'],
                'o-',
                color=PALETTE_MODELS.get(modelo, '#7F8C8D'),
                label=NOMBRES_MODELOS.get(modelo, modelo),
                linewidth=2,
                markersize=8
            )

    # Configuracion
    ax.set_xlabel('Umbral de Confianza', fontsize=11)
    ax.set_ylabel('IoU Promedio', fontsize=11)
    ax.set_title('Sensibilidad del Rendimiento al Umbral de Confianza',
                 fontsize=13, fontweight='bold')
    ax.legend(loc='best', fontsize=9)
    ax.set_ylim(0, 1.05)
    ax.set_xlim(0, 1)
    ax.xaxis.grid(True, linestyle='--', alpha=0.7)
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()

    # Guardar
    guardar_figura(fig, 'viz_13_sensibilidad_umbrales', output_dir, logger=logger)

    return fig

In [43]:
# =============================================================================
# FUNCION ORQUESTADORA
# =============================================================================

def ejecutar_bloque1(
    ruta_metricas: Path = None,
    ruta_pca: Path = None,
    output_dir: Path = None
) -> Dict[str, any]:
    """
    Ejecuta todas las visualizaciones del Bloque 1.

    Args:
        ruta_metricas: Ruta al archivo de metricas fusionadas
        ruta_pca: Ruta al archivo de componentes PCA
        output_dir: Directorio de salida

    Returns:
        Diccionario con resultados y rutas de figuras
    """
    # Configurar logging
    logger = configurar_logging('Fase2E_Bloque1')

    logger.info("=" * 60)
    logger.info("FASE 2E - BLOQUE 1: VISUALIZACIONES ESTADISTICAS")
    logger.info("=" * 60)

    # Aplicar estilos
    aplicar_estilo()

    # Resolver rutas
    if ruta_metricas is None:
        ruta_metricas = ARCHIVOS['metricas_fusionadas']
    if ruta_pca is None:
        ruta_pca = ARCHIVOS['pca_componentes']
    if output_dir is None:
        output_dir = RUTAS['output_bloque1']

    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Cargar datos
    logger.info("\nCargando datos...")

    try:
        df = pd.read_csv(ruta_metricas)
        logger.info(f"Metricas cargadas: {len(df)} registros")
    except Exception as e:
        logger.error(f"Error cargando metricas: {e}")
        return {'error': str(e)}

    # Agregar columna modelo_norm si no existe
    df = agregar_columna_modelo(df)

    # Cargar PCA si disponible
    df_pca = None
    try:
        if Path(ruta_pca).exists():
            df_pca = pd.read_csv(ruta_pca)
            logger.info(f"PCA cargado: {len(df_pca)} registros")
    except Exception as e:
        logger.warning(f"PCA no disponible: {e}")

    # Ejecutar visualizaciones
    logger.info("\nGenerando visualizaciones...")
    resultados = {'figuras': [], 'errores': []}

    visualizaciones = [
        ('viz_01', viz_01_boxplot_iou_modelos),
        ('viz_02', viz_02_violin_iou_modelos),
        ('viz_03', viz_03_heatmap_iou_modelo_foto),
        ('viz_04', viz_04_scatter_apertura_iou),
        ('viz_05', viz_05_scatter_complejidad_iou),
        ('viz_06', viz_06_scatter_contraste_iou),
        ('viz_07', viz_07_radar_modelos),
        ('viz_08', viz_08_ranking_top20),
        ('viz_09', viz_09_lineas_iou_fotos),
        ('viz_10', viz_10_heatmap_correlaciones),
        ('viz_12', viz_12_boxplot_arquitecturas),
        ('viz_13', viz_13_sensibilidad_umbrales),
    ]

    for nombre, func in visualizaciones:
        try:
            fig = func(df, output_dir, logger)
            resultados['figuras'].append(nombre)
            plt.close(fig)
        except Exception as e:
            logger.error(f"Error en {nombre}: {e}")
            resultados['errores'].append((nombre, str(e)))

    # PCA 3D (requiere datos adicionales)
    try:
        fig = viz_11_pca_3d_scatter(df, df_pca, output_dir, logger)
        resultados['figuras'].append('viz_11')
        plt.close(fig)
    except Exception as e:
        logger.error(f"Error en viz_11: {e}")
        resultados['errores'].append(('viz_11', str(e)))

    # Resumen
    logger.info("\n" + "=" * 60)
    logger.info("RESUMEN BLOQUE 1")
    logger.info("=" * 60)
    logger.info(f"Visualizaciones generadas: {len(resultados['figuras'])}/13")
    logger.info(f"Errores: {len(resultados['errores'])}")
    logger.info(f"Directorio de salida: {output_dir}")

    if resultados['errores']:
        logger.warning("\nErrores encontrados:")
        for nombre, error in resultados['errores']:
            logger.warning(f"  - {nombre}: {error}")

    return resultados

In [44]:
# =============================================================================
# PUNTO DE ENTRADA
# =============================================================================

if __name__ == "__main__":
    resultados = ejecutar_bloque1()
    print(f"\nFiguras generadas: {len(resultados.get('figuras', []))}")

[23:56:16] INFO - ============================================================


INFO:Fase2E_Bloque1:============================================================


[23:56:16] INFO - FASE 2E - BLOQUE 1: VISUALIZACIONES ESTADISTICAS


INFO:Fase2E_Bloque1:FASE 2E - BLOQUE 1: VISUALIZACIONES ESTADISTICAS


[23:56:16] INFO - ============================================================


...

[23:56:33] INFO - Errores: 0


INFO:Fase2E_Bloque1:Errores: 0


[23:56:33] INFO - Directorio de salida: /content/drive/MyDrive/TFM/3_Analisis/fase2e_visualizaciones/bloque1_estadisticas


INFO:Fase2E_Bloque1:Directorio de salida: /content/drive/MyDrive/TFM/3_Analisis/fase2e_visualizaciones/bloque1_estadisticas



Figuras generadas: 13


In [45]:
# Ejecuta esto en Colab para ver las columnas
# import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/TFM/3_Analisis/fase2b_correlaciones/metricas_fusionadas.csv')
# print("Columnas disponibles:")
# print(df.columns.tolist())